In [0]:
#!import "../DataModel/DataStructure"

In [0]:
using System.IO;
using System.Text;
using Systemorph.Vertex.DataSetReader.Csv;
using Systemorph.Vertex.Session;
using Systemorph.Vertex.Import.Builders;
using Systemorph.Vertex.Import.Mappings;
using Systemorph.Vertex.DataSetReader;

public record StreamWrapper(Stream Stream, bool WillBeReused);

public static string ProcessNotification(this object obj) => obj is ActivityMessageNotification amn ? amn.Message : ""; 

In [0]:
public record KeyedActivity : KeyedRecord
{
    public string Username {get; init;}

    public DateTime StartDateTime {get; init;}

    public DateTime EndDateTime {get; init;}

    public ActivityLogStatus Status {get; init;}

    public string Category {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] ErrorMessages {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] WarningMessages {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] InfoMessages {get; init;}

    public Guid SourceId {get; init;} // Check if converting to Guid causes issues. If not, leave Guid.

    public KeyedActivity(ActivityLog log, ISessionVariable session)
    {
        Id = Guid.NewGuid();
        Username = session.User.Name;
        StartDateTime = log.StartDateTime;
        EndDateTime = log.FinishDateTime;
        Status = log.Status;
        ErrorMessages = log.Errors.Select(x => x.ProcessNotification()).Distinct().ToArray();
        WarningMessages = log.Warnings.Select(x => x.ProcessNotification()).Distinct().ToArray();
        InfoMessages = log.Infos.Select(x => x.ProcessNotification()).Distinct().ToArray();
    }

}

In [0]:
public abstract record ImportWithKey : KeyedRecord
{
    public DateTime CreationTime {get; init;}
    public byte[] SerializedContent {get; init;}
    public uint Length {get; init;}
    public string Format {get; init;}
    private protected ImportOptions importOptions {get; init;}

    public async Task<ImportWithKey> InitializeImportDataAsync(ISessionVariable session)
    {
        var stream = await GenerateStreamWrapperAsync(session);
        var formatAndContent = await GetInformationFromStreamAsync(stream, session);
        return this with{CreationTime = DateTime.UtcNow, 
                            Format = formatAndContent.Format ?? importOptions.Format,
                            SerializedContent = formatAndContent.Content,
                            Length = (uint)SerializedContent.Length,
                        };
    }

    private async Task<StreamWrapper> GenerateStreamWrapperAsync(ISessionVariable session)
    {
        StreamWrapper stream = importOptions switch
        {
            FileImportOptions fio => new StreamWrapper(await fio.Storage.ReadAsync(fio.FileName, session.CancellationToken), true),
            StreamImportOptions streamImportOptions => new StreamWrapper(streamImportOptions.Stream, false),
            StringImportOptions stringImportOptions => new StreamWrapper(new MemoryStream(Encoding.ASCII.GetBytes(stringImportOptions.Content)), true),
            DataSetImportOptions dataSetImportOptions => new StreamWrapper(new MemoryStream(Encoding.ASCII.GetBytes(DataSetCsvSerializer.Serialize(dataSetImportOptions.DataSet))), true),
            _ => null
        };
        return stream;
    }

    private async Task<(string Format, byte[] Content)> GetInformationFromStreamAsync(StreamWrapper stream, ISessionVariable session)
    {
        byte[] content;
        string format;
        using (MemoryStream ms = new MemoryStream())
        {
            await stream.Stream.CopyToAsync(ms);
            content = ms.ToArray();
            ms.Position = 0;
            DataSetImportVariable importVariable = new DataSetImportVariable(session, sess => sess.CancellationToken);
            var dsRes = await importVariable.ReadFromStream(ms).ExecuteAsync();
            format = dsRes.Format;
            if (stream.WillBeReused)
                stream.Stream.Position = 0;
            else
            {
                stream.Stream.Close();
                await stream.Stream.DisposeAsync();
            }
        }
        return (format, content);
    }
}

In [0]:
public record KeyedFile : ImportWithKey
{
    public string Name {get; init;}

    public string Directory {get; init;}
    
    public string ContentType {get; init;}

    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] Partition {get; init;}

    public string Source {get; init;}

    public KeyedFile(FileImportOptions options)
    {
        importOptions = options;
        string fileName = options.FileName; 
        Id = Guid.NewGuid();
        Name = Path.GetFileName(fileName);
        Directory = Path.GetDirectoryName(fileName); 
        ContentType = Path.GetExtension(fileName);
        Source = options.Storage.GetType().Name; 
        Partition =  GetInvolvedPartitions(options);
        // Andrey Katz: Options.TargetDataSource.Partion.GetCurrent(?? What do we put here, different classes might posess various partitions, e.g. Yield Curve has none ??)
    }

    private string[] GetInvolvedPartitions(ImportOptions options)
    {
        // TODO
        //Andrey Katz: Get all the relevant partitions here 
        return null;
    }
}

In [0]:
public record KeyedStringImport : ImportWithKey
{
    public string Content {get; init;}

    public KeyedStringImport(StringImportOptions options)
    {
        importOptions = options;
        Id = Guid.NewGuid();
        Content = options.Content;
    }
}

In [0]:
public record KeyedDataSetImport : ImportWithKey
{
    public KeyedDataSetImport(DataSetImportOptions options)
    {
        importOptions = options;
        Id = Guid.NewGuid();
    }
}

In [0]:
public record KeyedStreamImport : ImportWithKey
{
    public KeyedStreamImport(StreamImportOptions options)
    {
        importOptions = options;
        Id = Guid.NewGuid();
    }
}

In [0]:
public static async Task<ActivityLog> ExecuteWithStoreActivityAsync(this ImportOptionsBuilder builder, 
                                                                                ISessionVariable session, 
                                                                                IDataSource dataSource)
{
    var log = await builder.ExecuteAsync();
    var options = builder.GetImportOptions();
    var activity = new KeyedActivity(log, session);
    switch(options)
    {
        case FileImportOptions fio:
            var keyedFile = await (new KeyedFile(fio)).InitializeImportDataAsync(session);
            activity = activity with {SourceId = keyedFile.Id, 
                                        Category = "Import from File"};
            await dataSource.UpdateAsync<KeyedFile>((keyedFile as KeyedFile).RepeatOnce());
            break;
        case StringImportOptions sgio:
            var keyedStringImport = await (new KeyedStringImport(sgio)).InitializeImportDataAsync(session);
            activity = activity with {SourceId = keyedStringImport.Id, 
                                    Category = "Import from String"};
            await dataSource.UpdateAsync<KeyedStringImport>((keyedStringImport as KeyedStringImport).RepeatOnce());
            break;
        case StreamImportOptions smio:
            var keyedStreamImport = await (new KeyedStreamImport(smio)).InitializeImportDataAsync(session);
            activity = activity with {SourceId = keyedStreamImport.Id, 
                                    Category = "Import from Stream"};
            await dataSource.UpdateAsync<KeyedStreamImport>((keyedStreamImport as KeyedStreamImport).RepeatOnce());
            break;
        case DataSetImportOptions dsio:
            var keyedDataSetImport = await (new KeyedDataSetImport(dsio)).InitializeImportDataAsync(session);
            activity = activity with {SourceId = keyedDataSetImport.Id, 
                                    Category = "Import from Data Set"};
            await dataSource.UpdateAsync<KeyedDataSetImport>((keyedDataSetImport as KeyedDataSetImport).RepeatOnce());
            break;
        default:
            break;
    }                              
    await dataSource.UpdateAsync<KeyedActivity>(activity.RepeatOnce());
    await dataSource.CommitAsync(); 
    return log;
}